In [ ]:
val df = {
    spark
    .read
    .schema("vendor_id string, pickup_datetime timestamp, dropoff_datetime timestamp, passenger_count integer, trip_distance double, pickup_lon double, pickup_lat double, rate_code integer, store_and_fwd_flag string, dropoff_lon double, dropoff_lat double, payment_type string, fare_amount double, surcharge double, mta_tax double, tip_amount double, tolls_amount double, total_amount double")
    .format("csv")
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .load("s3://nyc-tlc/trip data/yellow_tripdata_201[0-4]-*.csv")
}

In [ ]:
val cleaned = { 
    df
    .filter($"pickup_datetime" >= "2010-01-01T00:00:00.000Z")
    .filter($"dropoff_datetime" < "2015-01-01T00:00:00.000Z")
    .orderBy("dropoff_datetime")
    .withColumn("trip_id", monotonically_increasing_id())
    .withColumn("type", lit("trip"))
}

In [ ]:
cleaned.printSchema

In [ ]:
cleaned.write.json("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.json/")

In [ ]:
cleaned.write.option("compression", "snappy").json("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.json.snappy/")

In [ ]:
cleaned.write.option("compression", "gzip").json("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.json.gz/")

In [ ]:
cleaned.write.option("compression", "lz4").json("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.json.lz4/")

In [ ]:
cleaned.write.parquet("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.parquet/")

In [ ]:
cleaned.write.orc("s3://shausma-nyc-tlc/trip-data-raw/taxi-trips.orc/")